<a href="https://colab.research.google.com/github/YasJanam/RL1/blob/main/REINFORCE_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gym
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=

In [2]:
env = gym.make("CartPole-v1")

/usr/local/lib/python3.12/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.12/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [36]:
print(env.observation_space)
print(env.action_space)
print(env.observation_space.shape)

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
Discrete(2)
(4,)


In [16]:
class PolicyNet(nn.Module):
  def __init__(self, state_dim, action_dim,hidden_dim=128):
    super(PolicyNet, self).__init__()
    self.fc = nn.Sequential(
        nn.Linear(state_dim, hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim, action_dim),
        nn.Softmax(dim=-1)
    )

  def forward(self, x):
    return self.fc(x)

In [18]:
def compute_returns(rewards, gamma):
  G = 0
  returns = []
  for r in reversed(rewards):
    G = r + gamma * G
    returns.insert(0,G)
  returns = torch.tensor(returns)
  returns = (returns - returns.mean()) / (returns.std() + 1e-8) # normalization
  return returns

In [21]:
def getPolicy(env=env,hidden_dim=2):
  state_dim = env.observation_space.shape[0]
  action_dim = env.action_space.n
  policy = PolicyNet(state_dim, action_dim,hidden_dim=hidden_dim)
  return policy

In [33]:
def TrainFunc(hidd_dim,lr=0.01,gamma=0.99,episode_num=1000):
  policy = getPolicy(hidden_dim=hidd_dim)  # hidd_dim => Policy_Net_hidden_dim
  optimizer = optim.Adam(policy.parameters(), lr=lr)
  for episode in range(episode_num):
    state = env.reset()
    log_probs = []
    rewards = []
    done = False

    # rollout
    while not done:
      state = torch.tensor(state, dtype=torch.float32)

      probs = policy(state)
      dist = Categorical(probs)
      action = dist.sample()
      log_prob = dist.log_prob(action)

      next_state, reward, done, _ = env.step(action.item())

      log_probs.append(log_prob)
      rewards.append(reward)
      state = next_state

    # policy update
    returns = compute_returns(rewards, gamma)

    loss = 0
    for log_prob, G in zip(log_probs, returns):
      loss -= log_prob * G

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_reward = sum(rewards)
    if episode % 100 == 0 :
      print(f"Episode {episode}, Total Reward: {total_reward}")

  env.close()
  return policy

##### train

برابر 500 هستش CartPole-v1 بیشترین امتیاز قابل دریافت در

طوری بشه که به این امتیاز برسیم و حفظش کنه policy باید

In [27]:
polic = TrainFunc(hidd_dim=3,lr=0.01,gamma=0.99,episode_num=1000)  # lr -> 0.01

Episode 0, Total Reward: 18.0
Episode 100, Total Reward: 23.0
Episode 200, Total Reward: 55.0
Episode 300, Total Reward: 222.0
Episode 400, Total Reward: 249.0
Episode 500, Total Reward: 486.0
Episode 600, Total Reward: 500.0
Episode 700, Total Reward: 500.0
Episode 800, Total Reward: 500.0
Episode 900, Total Reward: 500.0


In [28]:
polic = TrainFunc(hidd_dim=3,lr=0.001,gamma=0.99,episode_num=1000)  # lr -> 0.001

Episode 0, Total Reward: 20.0
Episode 100, Total Reward: 12.0
Episode 200, Total Reward: 20.0
Episode 300, Total Reward: 13.0
Episode 400, Total Reward: 12.0
Episode 500, Total Reward: 17.0
Episode 600, Total Reward: 54.0
Episode 700, Total Reward: 16.0
Episode 800, Total Reward: 20.0
Episode 900, Total Reward: 26.0
